<a href="https://colab.research.google.com/github/Linar23/Research_work/blob/master/Research_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!unzip /content/Graph

Archive:  /content/Graph.zip
   creating: content/Graph/
   creating: content/Graph/acc_train/
  inflating: content/Graph/acc_train/events.out.tfevents.1559350338.18ff2474c310  
   creating: content/Graph/avg_loss_train/
  inflating: content/Graph/avg_loss_train/events.out.tfevents.1559350338.18ff2474c310  
   creating: content/Graph/avg_mask_loss_test/
  inflating: content/Graph/avg_mask_loss_test/events.out.tfevents.1559350357.18ff2474c310  
   creating: content/Graph/avg_next_loss_test/
  inflating: content/Graph/avg_next_loss_test/events.out.tfevents.1559350357.18ff2474c310  
   creating: content/Graph/acc_test/
  inflating: content/Graph/acc_test/events.out.tfevents.1559350357.18ff2474c310  
   creating: content/Graph/avg_next_loss_train/
  inflating: content/Graph/avg_next_loss_train/events.out.tfevents.1559350338.18ff2474c310  
   creating: content/Graph/avg_mask_loss_train/
  inflating: content/Graph/avg_mask_loss_train/events.out.tfevents.1559350338.18ff2474c310  
   creating:

In [0]:
!zip -r /content/Graph.zip /content/Graph

  adding: content/Graph/ (stored 0%)
  adding: content/Graph/avg_mask_loss_train/ (stored 0%)
  adding: content/Graph/avg_mask_loss_train/events.out.tfevents.1560472127.012d5558c992 (deflated 47%)
  adding: content/Graph/avg_loss_test/ (stored 0%)
  adding: content/Graph/avg_loss_test/events.out.tfevents.1560472153.012d5558c992 (deflated 43%)
  adding: content/Graph/acc_train/ (stored 0%)
  adding: content/Graph/acc_train/events.out.tfevents.1560472127.012d5558c992 (deflated 41%)
  adding: content/Graph/avg_next_loss_train/ (stored 0%)
  adding: content/Graph/avg_next_loss_train/events.out.tfevents.1560472127.012d5558c992 (deflated 47%)
  adding: content/Graph/avg_next_loss_test/ (stored 0%)
  adding: content/Graph/avg_next_loss_test/events.out.tfevents.1560472153.012d5558c992 (deflated 46%)
  adding: content/Graph/avg_mask_loss_test/ (stored 0%)
  adding: content/Graph/avg_mask_loss_test/events.out.tfevents.1560472153.012d5558c992 (deflated 46%)
  adding: content/Graph/acc_test/ (stor

In [0]:
!mkdir /content/content/Graph

In [0]:
!pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab
tb = TensorBoardColab(graph_path='/content/Graph')

Wait for 8 seconds...
TensorBoard link:
https://7e915215.ngrok.io


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!rm -r -f /content/Graph.zip

In [0]:
import torch
import random
from collections import Counter
from math import sqrt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import math
import numpy as np
import pickle


class BERTDataset:
    def __init__(self, corpus_path, vocab, seq_len):
        """
        :param corpus_path:
        :param vocab:
        :param seq_len:
        """
        self.vocab = vocab
        self.seq_len = seq_len

        self.corpus_path = corpus_path

        with open(corpus_path, "r") as f:
            self.lines = [line.replace("\n", "").split("\\t") for line in f]

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, item):
        t1, t2, is_next_label = self.random_sent(item)
        t1_random, t1_label = self.random_word(t1)
        t2_random, t2_label = self.random_word(t2)

        t1 = [self.vocab.cls_index] + t1_random + [self.vocab.sep_index]
        t2 = t2_random + [self.vocab.sep_index]

        t1_label = [self.vocab.pad_index] + t1_label + [self.vocab.pad_index]
        t2_label = t2_label + [self.vocab.pad_index]

        segment_label = ([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:self.seq_len]
        bert_input = (t1 + t2)[:self.seq_len]
        bert_label = (t1_label + t2_label)[:self.seq_len]

        padding = [self.vocab.pad_index for _ in range(self.seq_len - len(bert_input))]

        bert_input += padding
        bert_label += padding
        segment_label += padding

        output = {"bert_input": bert_input,
                  "bert_label": bert_label,
                  "segment_label": segment_label,
                  "is_next": is_next_label}

        return {key: torch.tensor(value) for key, value in output.items()}

    def random_word(self, sentence):
        tokens = sentence.split()
        output_label = []

        for i, token in enumerate(tokens):
            prob = random.random()
            if prob < 0.15:
                prob /= 0.15

                if prob < 0.8:
                    tokens[i] = self.vocab.mask_index
                elif prob < 0.9:
                    tokens[i] = random.randrange(len(self.vocab))
                else:
                    tokens[i] = self.vocab.token_to_index.get(token, self.vocab.unk_index)

                output_label.append(self.vocab.token_to_index.get(token, self.vocab.unk_index))
            else:
                tokens[i] = self.vocab.token_to_index.get(token, self.vocab.unk_index)
                output_label.append(self.vocab.pad_index)

        return tokens, output_label

    def random_sent(self, index):
        t1, t2 = self.get_corpus_line(index)

        if random.random() > 0.5:
            return t1, t2, 1
        else:
            return t1, self.get_random_line(), 0

    def get_corpus_line(self, item):
        return self.lines[item][0], self.lines[item][1]

    def get_random_line(self):
        return self.lines[random.randrange(len(self.lines))][1]


class Vocab:
    def __init__(self, text):
        """
        :param text:
        """
        self.specials = ["<pad>", "<unk>", "<sep>", "<cls>", "<mask>"]

        self.pad_index = 0
        self.unk_index = 1
        self.sep_index = 2
        self.cls_index = 3
        self.mask_index = 4

        self.index_to_token = list(self.specials)

        counter = Counter()
        
        for line in text:
            words = line.replace("\n", "").replace("\\t", "").split()

            for word in words:
                counter[word] += 1

        words_and_frequencies = sorted(counter.items())

        for word, freq in words_and_frequencies:
            if(freq > 200):
                self.index_to_token.append(word)

        self.token_to_index = {token: i for i, token in enumerate(self.index_to_token)}

    def __len__(self):
        return len(self.index_to_token)


class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        """
        :param d_k: int scaling factor
        """
        super(ScaledDotProductAttention, self).__init__()

        self.scaling = 1 / (sqrt(d_k))

    def forward(self, q, k, v, mask):
        """
        :param q:  An float tensor with shape of [b_s, seq_len, d_model / n_head]
        :param k: An float tensor with shape of [b_s, seq_len, d_model / n_head]
        :param v: An float tensor with shape of [b_s, seq_len, d_model / n_head]

        :return: An float tensor with shape of [b_s, seq_len, d_model / n_head]
        """
        attention = torch.bmm(q, k.transpose(1, 2)) * self.scaling
        attention = attention.masked_fill(mask == 0, -1e9)

        attention = F.softmax(attention, dim=2)

        output = torch.bmm(attention, v)

        return output


class SingleHeadAttention(nn.Module):
    def __init__(self, d_model, d_k, d_v):
        """
        :param d_model: Int
        :param d_k: Int = d_model / n_head
        :param d_v: Int = d_model / n_head
        """
        super(SingleHeadAttention, self).__init__()

        self.q_linear = nn.Linear(d_model, d_k)
        self.k_linear = nn.Linear(d_model, d_k)
        self.v_linear = nn.Linear(d_model, d_v)

        self.attention = ScaledDotProductAttention(d_k)

    def forward(self, q, k, v, mask):
        """
        :param q: An float tensor with shape of [b_s, seq_len, d_model]
        :param k: An float tensor with shape of [b_s, seq_len, d_model]
        :param v: An float tensor with shape of [b_s, seq_len, d_model]

        :return: An float tensor with shape of [b_s, seq_len, d_model / n_heads]
        """
        proj_q = self.q_linear(q)
        proj_k = self.k_linear(k)
        proj_v = self.v_linear(v)

        output = self.attention(proj_q, proj_k, proj_v, mask)

        return output


class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, d_model):
        """
        :param n_head: Int number of heads
        :param d_model: Int
        """
        super(MultiHeadAttention, self).__init__()

        d_v = int(d_model / n_head)
        d_k = int(d_model / n_head)

        self.attention = nn.ModuleList([SingleHeadAttention(d_model, d_k, d_v) for _ in range(n_head)])

        self.Linear = nn.Linear(n_head * d_v, d_model)

    def forward(self, q, k, v, mask):
        """
        :param q: An float tensor with shape of [b_s, seq_len, d_model]
        :param k: An float tensor with shape of [b_s, seq_len, d_model]
        :param v: An float tensor with shape of [b_s, seq_len, d_model]

        :return: An float tensor with shape of [b_s, seq_len, d_model]
        """
        results = []

        for i, single_attention in enumerate(self.attention):
            attention_out = single_attention(q, k, v, mask)
            results.append(attention_out)

        concat = torch.cat(results, dim=2)

        linear_output = self.Linear(concat)

        return linear_output


class TokenEmbedding(nn.Embedding):
    def __init__(self, vocab_size, emb_size):
        super().__init__(vocab_size, emb_size)


class SegmentEmbedding(nn.Embedding):
    def __init__(self, emb_size):
        super().__init__(3, emb_size)


class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()

        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = torch.pow(10000, torch.arange(0, d_model, 2).float() / d_model)

        pe[:, 0::2] = torch.sin(position / div_term)
        pe[:, 1::2] = torch.cos(position / div_term)

        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return self.pe[:, :x.size(1)].to('cuda')


class BERTEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        """
        :param vocab_size: Int size of vocabulary
        :param emb_size: Int size of embedding
        """
        super(BERTEmbedding, self).__init__()

        self.v_s = vocab_size
        self.e_s = emb_size

        self.token = TokenEmbedding(self.v_s, self.e_s)
        self.segment = SegmentEmbedding(self.e_s)
        self.position = PositionalEmbedding(self.e_s)

    def forward(self, seq, segment_label):
        """
        :param seq: An long tensor with shape of [b_s, seq_len]

        :return: An float tensor with shape of [b_s, seq_len, emb_size]
        """

        return self.token(seq) + self.segment(segment_label) + self.position(seq)


class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


class PositionWise(nn.Module):
    def __init__(self, size, inner_size):
        """
        :param size: Int input size
        :param inner_size: Int inner size of position wise
        """
        super(PositionWise, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(size, inner_size),
            GELU(),
            nn.Linear(inner_size, size)
        )

        self.layer_norm = nn.LayerNorm(size)

    def forward(self, input):
        """
        :param input: An float tensor with shape of [b_s, seq_len, emb_size]

        :return: An float tensor with shape of [b_s, seq_len, emb_size]
        """
        residual = input

        result = self.fc(input)

        return self.layer_norm(result + residual)


class Encoder(nn.Module):
    def __init__(self, embeddings, d_model, n_heads, n_layers, vocab_s):
        """
        :param embeddings: An float embeddings tensor with shape [b_s, seq_len, d_model]
        :param d_model: Int size of input
        :param n_heads: Int number of heads
        :param vocab_s: Int size of vocabulary
        """
        super(Encoder, self).__init__()

        self.embeddings = embeddings
        self.vocab_s = vocab_s

        self.transformer_blocks = nn.ModuleList([nn.Sequential(MultiHeadAttention(n_heads, d_model), nn.LayerNorm(d_model), PositionWise(d_model, d_model * 4)) for _ in range(n_layers)])


    def forward(self, x, segment_label):
        """
        :param input: An long tensor with shape of [b_s, seq_len]

        :return: An float tensor with shape of [b_s, seq_len, vocab_size]
        """

        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1)
        input = self.embeddings(x, segment_label)

        for multi_head_block, layer_norm, position_wise in self.transformer_blocks:
            input = layer_norm(input + multi_head_block(q=input, k=input, v=input, mask=mask))
            input = position_wise(input)

        return input


class Model(nn.Module):
    def __init__(self, n_heads, n_layers, vocab_size, emb_size):
        """
        :param n_heads: Int number of heads
        :param vocab_size: Int size of vocabulary
        :param emb_size: Int embedding size
        """
        super(Model, self).__init__()

        self.embed = BERTEmbedding(vocab_size, emb_size)

        self.d_model = self.embed.e_s
        self.v_s = self.embed.v_s

        self.encoder = Encoder(self.embed, self.d_model, n_heads, n_layers, self.v_s)
        self.next_sentence = NextSentencePrediction(self.d_model)
        self.mask_lm = MaskedLanguageModel(self.d_model, self.v_s)

    def forward(self, x, segment_label):
        """
        :param x: An float tensor with shape of [b_s, seq_len]
        :param segment_label: An float tensor with shape of [b_s, seq_len]
        """
        prediction = self.encoder(x, segment_label)

        return self.next_sentence(prediction), self.mask_lm(prediction)


class NextSentencePrediction(nn.Module):
    def __init__(self, d_model):
        """
        :param d_model: Int
        """
        super().__init__()
        self.linear = nn.Linear(d_model, 2)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x[:, 0]))


class MaskedLanguageModel(nn.Module):
    def __init__(self, d_model, vocab_size):
        """
        :param d_model: Int
        :param vocab_size: Int size of vocabulary
        """
        super().__init__()
        self.linear = nn.Linear(d_model, vocab_size)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x))


class AdamW(torch.optim.Optimizer):
    """Implements AdamW algorithm.
    It has been proposed in `Fixing Weight Decay Regularization in Adam`_.
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-3)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square (default: (0.9, 0.999))
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
    .. Fixing Weight Decay Regularization in Adam:
    https://arxiv.org/abs/1711.05101
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay)
        super(AdamW, self).__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('AdamW does not support sparse gradients, please consider SparseAdam instead')

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # according to the paper, this penalty should come after the bias correction
                # if group['weight_decay'] != 0:
                #     grad = grad.add(group['weight_decay'], p.data)

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1

                # w = w - wd * lr * w
                if group['weight_decay'] != 0:
                    p.data.add_(-group['weight_decay'] * group['lr'], p.data)

                # w = w - lr * w.grad
                p.data.addcdiv_(-step_size, exp_avg, denom)

                # w = w - wd * lr * w - lr * w.grad
                # See http://www.fast.ai/2018/07/02/adam-weight-decay/

        return loss


class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''

    def __init__(self, optimizer, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        "Zero out the gradients by the inner optimizer"
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr
            
            
dataset_path = "/content/gdrive/My Drive/data_small.txt"

with open(dataset_path, "r") as f:
    vocab = Vocab(f)
    
#with open("/content/vocab_small.pickle", "wb") as f:
#    pickle.dump(vocab, f)
    
# with open("/content/dataset_small.pickle", "wb") as f:

#with open('/content/vocab_small.pickle', 'rb') as f:
#    vocab = pickle.load(f)
    
#with open('/content/dataset_small.pickle', 'rb') as f:
#   dataset = pickle.load(f)
        
print(len(vocab))

seq_len = 64
emb_size = 94
epochs = 10
n_layers = 4

dataset = BERTDataset(dataset_path, vocab, seq_len)

train, test = train_test_split(dataset, test_size=0.2)

train_data_loader = DataLoader(train, batch_size=128, shuffle=True)
test_data_loader = DataLoader(test, batch_size=128)
#model = torch.load("/content/model_epoch_7.pth").to('cuda')
model = Model(3, n_layers, len(vocab), emb_size).to('cuda')
masked_criterion = nn.CrossEntropyLoss(ignore_index=0)
next_criterion = nn.CrossEntropyLoss()
optim = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
optim_schedule = ScheduledOptim(optim, emb_size, n_warmup_steps=10000)

train_loss = []
test_loss = []
train_acc = []
test_acc = []

#writer = SummaryWriter()

for l in range(epochs):
    print("Train")

    avg_loss = 0.0
    avg_next_loss = 0.0
    avg_mask_loss = 0.0
    total_correct = 0
    total_element = 0

    for i, data in enumerate(train_data_loader):
        #print(i / len(train_data_loader))
        data = {key: value.to('cuda') for key, value in data.items()}

        next_sent_output, mask_lm_output = model.forward(data["bert_input"], data["segment_label"])

        next_loss = next_criterion(next_sent_output, data["is_next"])
        mask_loss = masked_criterion(mask_lm_output.transpose(1, 2), data["bert_label"])
        
        #print("Next loss = " + str(next_loss))
        #print("Mask loss = " + str(mask_loss))

        loss = next_loss + mask_loss

        correct = next_sent_output.argmax(dim=-1).eq(data["is_next"]).sum().item()
        
        avg_next_loss += next_loss.item()
        avg_mask_loss += mask_loss.item()
        avg_loss += loss.item()
        total_correct += correct
        total_element += data["is_next"].nelement()

        #print("Epoch " + str(l) + " avg_loss = " + str(avg_loss / (i + 1)) + " total_acc = " + str(total_correct * 100.0 / total_element))

        optim_schedule.zero_grad()
        loss.backward()
        optim_schedule.step_and_update_lr()

    train_loss.append(avg_loss)
    train_acc.append(total_correct * 100.0 / total_element)
    
    print('Avg loss train ' + str(avg_loss / (i + 1)) + " epoch " + str(l))
    print('Acc train ' + str(total_correct * 100.0 / total_element) + " epoch " +  str(l))
   
    tb.save_value('Avg loss train', 'avg_loss_train', l, avg_loss / (i + 1))
    tb.save_value('Avg next loss train', 'avg_next_loss_train', l, avg_next_loss / (i + 1))
    tb.save_value('Avg mask loss train', 'avg_mask_loss_train', l, avg_mask_loss / (i + 1))
    tb.save_value('Acc train', 'acc_train', l, total_correct * 100.0 / total_element)

    avg_loss = 0.0
    avg_next_loss = 0.0
    avg_mask_loss = 0.0
    total_correct = 0
    total_element = 0

    print("Test")

    for i, data in enumerate(test_data_loader):
        data = {key: value.to('cuda') for key, value in data.items()}

        next_sent_output, mask_lm_output = model.forward(data["bert_input"], data["segment_label"])
                                                                         
        next_loss = next_criterion(next_sent_output, data["is_next"])
        mask_loss = masked_criterion(mask_lm_output.transpose(1, 2), data["bert_label"])

        loss = next_loss + mask_loss
        loss = next_loss + mask_loss

        correct = next_sent_output.argmax(dim=-1).eq(data["is_next"]).sum().item()
        
        avg_next_loss += next_loss.item()
        avg_mask_loss += mask_loss.item()
        avg_loss += loss.item()
        total_correct += correct
        total_element += data["is_next"].nelement()

        #print("Epoch " + str(l) + " avg_loss = " + str(avg_loss / (i + 1)) + " total_acc = " + str(total_correct * 100.0 / total_element))

    test_loss.append(avg_loss)
    test_acc.append(total_correct * 100.0 / total_element)
    
    print('Avg loss test ' + str(avg_loss / (i + 1)) + " epoch " + str(l))
    print('Acc test ' + str(total_correct * 100.0 / total_element) + " epoch " +  str(l))
    
    tb.save_value('Avg loss test', 'avg_loss_test', l, avg_loss / (i + 1))
    tb.save_value('Avg next loss test', 'avg_next_loss_test', l, avg_next_loss / (i + 1))
    tb.save_value('Avg mask loss test', 'avg_mask_loss_test', l, avg_mask_loss / (i + 1))
    tb.save_value('Acc test', 'acc_test', l, total_correct * 100.0 / total_element)
    
    #torch.save(model.cuda(), "model_epoch_" + str(l))
    
    print()

6353
Train
Avg loss train 9.339153177185059 epoch 0
Acc train 52.09788024470061 epoch 0
Test
Avg loss test 9.291124470703437 epoch 0
Acc test 54.484 epoch 0

Train
Avg loss train 9.260304146728515 epoch 1
Acc train 60.10615026537566 epoch 1
Test
Avg loss test 9.240381749389726 epoch 1
Acc test 63.052 epoch 1

Train
Avg loss train 9.202615212402344 epoch 2
Acc train 66.85941714854287 epoch 2
Test
Avg loss test 9.179325451326493 epoch 2
Acc test 69.22 epoch 2

Train
Avg loss train 9.1598949899292 epoch 3
Acc train 70.57167642919107 epoch 3
Test
Avg loss test 9.148575881558001 epoch 3
Acc test 71.348 epoch 3

Train
Avg loss train 9.10768 epoch 4
Acc train 74.3831859579649 epoch 4
Test
Avg loss test 9.082255093947701 epoch 4
Acc test 75.705 epoch 4

Train


KeyboardInterrupt: ignored

In [0]:
!zip -r /content/Graph.zip /content/Graph

from google.colab import files
files.download("/content/Graph.zip")

updating: content/Graph/ (stored 0%)
updating: content/Graph/acc_train/ (stored 0%)
updating: content/Graph/avg_loss_train/ (stored 0%)
updating: content/Graph/avg_mask_loss_test/ (stored 0%)
updating: content/Graph/avg_next_loss_test/ (stored 0%)
updating: content/Graph/acc_test/ (stored 0%)
updating: content/Graph/avg_next_loss_train/ (stored 0%)
updating: content/Graph/avg_mask_loss_train/ (stored 0%)
updating: content/Graph/avg_loss_test/ (stored 0%)
  adding: content/Graph/acc_train/events.out.tfevents.1559358781.bad86040ca6c (deflated 48%)
  adding: content/Graph/avg_loss_train/events.out.tfevents.1559358781.bad86040ca6c (deflated 52%)
  adding: content/Graph/avg_mask_loss_test/events.out.tfevents.1559358808.bad86040ca6c (deflated 57%)
  adding: content/Graph/avg_next_loss_test/events.out.tfevents.1559358808.bad86040ca6c (deflated 54%)
  adding: content/Graph/acc_test/events.out.tfevents.1559358808.bad86040ca6c (deflated 51%)
  adding: content/Graph/avg_next_loss_train/events.out

In [0]:
!nvidia-smi

Sat Sep 14 11:13:49 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip3 install transformers

     |████████████████████████████████| 317kB 6.5MB/s 
     |████████████████████████████████| 860kB 44.0MB/s 
     |████████████████████████████████| 645kB 37.6MB/s 
     |████████████████████████████████| 1.0MB 40.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=d23566e6c01906de8500c525009b0e8961bfc61885057a18d0a09b07a43b393b
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
import torch
from transformers import *

# Transformers has a unified API
# for 8 transformer architectures and 30 pretrained weights.
#          Model          | Tokenizer          | Pretrained weights shortcut
MODELS = [
          (DistilBertForMaskedLM, DistilBertTokenizer, 'distilbert-base-uncased')]

# To use TensorFlow 2.0 versions of the models, simply prefix the class names with 'TF', e.g. `TFRobertaModel` is the TF 2.0 counterpart of the PyTorch model `RobertaModel`

# Let's encode some text in a sequence of hidden-states using each model:
for model_class, tokenizer_class, pretrained_weights in MODELS:
    # Load pretrained model/tokenizer
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

    # Encode text
    input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    with torch.no_grad():
        last_hidden_states = model(input_ids)  # Models outputs are now tuples
        print(last_hidden_states[0])

tensor([[[ -6.0730,  -6.0184,  -6.0532,  ...,  -5.3471,  -5.1327,  -3.2758],
         [-11.2493, -11.0514, -11.2565,  ..., -10.8661,  -9.0513,  -7.8534],
         [-12.1650, -11.9065, -12.0352,  ..., -10.5904,  -8.4455,  -7.9540],
         ...,
         [-12.6242, -12.2914, -12.5036,  ..., -11.2858, -10.2073, -11.0556],
         [ -8.3184,  -8.1826,  -8.2792,  ...,  -7.9680,  -7.4850,  -8.3284],
         [-11.3067, -11.3019, -11.3750,  ...,  -9.1290,  -9.4283,  -7.6095]]])


In [0]:
len(last_hidden_states[0][0][0])

30522

In [0]:
input_ids

tensor([[  101,  2182,  2003,  2070,  3793,  2000,  4372, 16044,   102]])